In [1]:
import random

In [3]:
class tank:
    pass

class ATS:
    pass

class battlefiend:
    def __init__(self, deep = 1):
        self.dct = {
            'Видимость': 1,
            'Влажность': 0,
            'Температура': 273
        }
        self.x, self.y = 100, 100
        self.spawn_x_1, self.spawn_x_2 = 0, 0
        self.spawn_y_1, self.spawn_y_2 = self.x, self.y

    def generate(self):
        self.spawn_x_1 = random.randint(0, self.l)
        self.spawn_y_1 = random.randint(0, self.l)
        self.spawn_x_2 = random.randint(0, self.l)
        self.spawn_y_2 = random.randint(0, self.l)

class figth:
    def __init__(self, tank, ATS):
        self.tank = tank
        self.ATS = ATS